#### Imports

In [1]:
from os import chdir
import matplotlib.pyplot as plt
%matplotlib inline
chdir('../')

from src import PrepareFreqTimeImage,PipelineImageToMask
from src import ImageToMaskDataset
from src import UNet,CNN1D,FilterCNN
from src import TrainImageToMaskNetworkModel
from src import PipelineImageToFilterToCCtoLabels, PipelineImageToFilterDelGraphtoIsPulsar,PipelineImageToCCtoLabels
from src import Tuner


##### Set the Notebook settings

In [2]:
need_data_generation:bool = False
need_training:bool = False


#### Set the loaders

In [3]:
image_preprocessing_engine = PrepareFreqTimeImage(
                                                do_rot_phase_avg=True,
                                                do_binarize=False,
                                                do_resize=True,
                                                resize_size=(128,128),
                                                )
mask_preprocessing_engine = PrepareFreqTimeImage(
                                                do_rot_phase_avg=True,
                                                do_binarize=True,
                                                do_resize=True,
                                                resize_size=(128,128),
                                                #BinarizeToMask(binarize_func='gaussian_blur') # or 'exponential'
                                                )

cnn_model_to_make_mask_path: str = './syn_data/model/trained_UNet_test_v0.pt'
mask_maker_engine = PipelineImageToMask(
                                image_to_mask_network=UNet(),
                                trained_image_to_mask_network_path=cnn_model_to_make_mask_path,                     
                                )



#### Generate dataset and train neural net with different data types

- Only Pulsar

In [4]:
from pulsar_simulation.generate_data_pipeline import generate_example_payloads_for_training

#: Run this node only if training and test data needs to be generated
if need_data_generation:
    generate_example_payloads_for_training(tag='train_only_pulsar_v0_',
                                        num_payloads=1000,
                                        plot_a_example=False,
                                        param_folder='./syn_data/runtime/',
                                        payload_folder='./syn_data/payloads/',
                                        num_cpus=10, #: choose based on the number of nodes/cores in your system,
                                        prob_bbrfi=0,
                                        prob_nbrfi=0
                                        )

    generate_example_payloads_for_training(tag='test_only_pulsar_v0_',
                                        num_payloads=1000,
                                        plot_a_example=False,
                                        param_folder='./syn_data/runtime/',
                                        payload_folder='./syn_data/payloads/',
                                        num_cpus=10, #: choose based on the number of nodes/cores in your system,
                                        prob_bbrfi=0,
                                        prob_nbrfi=0
                                        )

- Set up the datasets for training

In [ ]:
image_tag='train_only_pulsar_v0_*_payload_detected.json' #: '*' in the name is the index place holder of a image in the image set
image_directory='./syn_data/payloads/'

mask_tag = 'train_only_pulsar_v0_*_payload_flux.json'
mask_directory='./syn_data/payloads/'

image_mask_train_datasetPulsar = ImageToMaskDataset(
                        image_tag = image_tag,
                        mask_tag= mask_tag,
                        image_directory = image_directory,
                        mask_directory = mask_directory,
                        image_engine=image_preprocessing_engine,
                        mask_engine=mask_preprocessing_engine
                        )

image_tag='test_only_pulsar_v0_*_payload_detected.json' #: '*' in the name is the index place holder of a image in the image set
image_directory='./syn_data/payloads/'

mask_tag = 'test_only_pulsar_v0_*_payload_flux.json'
mask_directory='./syn_data/payloads/'

image_mask_test_datasetPulsar = ImageToMaskDataset(
                        image_tag = image_tag,
                        mask_tag= mask_tag,
                        image_directory = image_directory,
                        mask_directory = mask_directory,
                        image_engine=image_preprocessing_engine,
                        mask_engine=mask_preprocessing_engine
                        )
#: Plot an example
idx = 3
image_mask_train_datasetPulsar.plot(index=idx)
image_mask_test_datasetPulsar.plot(index=idx)


- instantiate the trainer

In [6]:
from src.pulsar_analysis.neural_network_models import  WeightedBCELoss
store_trained_model_image2mask_atPulsar = './syn_data/model/trained_UNet_test_only_pulsar_v0.pt'
image2mask_network_trainerPulsar = TrainImageToMaskNetworkModel(
                                model=UNet(),
                                num_epochs=10,
                                store_trained_model_at=store_trained_model_image2mask_atPulsar,
                                loss_criterion = WeightedBCELoss(pos_weight=3,neg_weight=1)                                
                                )

- train

In [7]:
if need_training: image2mask_network_trainerPulsar(image_mask_pairset=image_mask_train_datasetPulsar)

- Test

In [ ]:
idx = 0
image = image_mask_test_datasetPulsar[idx][0]
pred = image2mask_network_trainerPulsar.test_model(image=image,plot_pred=True)

- Only NBRFI

In [9]:
from pulsar_simulation.generate_data_pipeline import generate_example_payloads_for_training

#: Run this node only if training and test data needs to be generated
if need_data_generation:
    generate_example_payloads_for_training(tag='train_only_NBRFI_v0_',
                                        num_payloads=1000,
                                        plot_a_example=False,
                                        param_folder='./syn_data/runtime/',
                                        payload_folder='./syn_data/payloads/',
                                        num_cpus=10, #: choose based on the number of nodes/cores in your system,
                                        prob_bbrfi=0,
                                        prob_nbrfi=1,                                       
                                        antenna_sensitivity=100
                                        )

    generate_example_payloads_for_training(tag='test_only_NBRFI_v0_',
                                        num_payloads=1000,
                                        plot_a_example=False,
                                        param_folder='./syn_data/runtime/',
                                        payload_folder='./syn_data/payloads/',
                                        num_cpus=10, #: choose based on the number of nodes/cores in your system,
                                        prob_bbrfi=0,
                                        prob_nbrfi=1,                                       
                                        antenna_sensitivity=100
                                        )

- Set up the datasets for training

In [ ]:
image_tag='train_only_NBRFI_v0_*_payload_detected.json' #: '*' in the name is the index place holder of a image in the image set
image_directory='./syn_data/payloads/'

mask_tag = 'train_only_NBRFI_v0_*_payload_flux.json'
mask_directory='./syn_data/payloads/'

image_mask_train_datasetNBRFI = ImageToMaskDataset(
                        image_tag = image_tag,
                        mask_tag= mask_tag,
                        image_directory = image_directory,
                        mask_directory = mask_directory,
                        image_engine=image_preprocessing_engine,
                        mask_engine=mask_preprocessing_engine
                        )

image_tag='test_only_NBRFI_v0_*_payload_detected.json' #: '*' in the name is the index place holder of a image in the image set
image_directory='./syn_data/payloads/'

mask_tag = 'test_only_NBRFI_v0_*_payload_flux.json'
mask_directory='./syn_data/payloads/'

image_mask_test_datasetNBRFI = ImageToMaskDataset(
                        image_tag = image_tag,
                        mask_tag= mask_tag,
                        image_directory = image_directory,
                        mask_directory = mask_directory,
                        image_engine=image_preprocessing_engine,
                        mask_engine=mask_preprocessing_engine
                        )
#: Plot an example
idx = 8
image_mask_train_datasetNBRFI.plot(index=idx)
image_mask_test_datasetNBRFI.plot(index=idx)


- instantiate the trainer

In [11]:
from src.pulsar_analysis.neural_network_models import  WeightedBCELoss
store_trained_model_image2mask_atNBRFI = './syn_data/model/trained_UNet_test_only_NBRFI_v0.pt'
image2mask_network_trainerNBRFI = TrainImageToMaskNetworkModel(
                                model=UNet(),
                                num_epochs=10,
                                store_trained_model_at=store_trained_model_image2mask_atNBRFI,
                                loss_criterion = WeightedBCELoss(pos_weight=3,neg_weight=1)                                
                                )

- train

In [12]:
if need_training:image2mask_network_trainerNBRFI(image_mask_pairset=image_mask_train_datasetNBRFI)

- Test

In [ ]:
idx = 0
image = image_mask_test_datasetNBRFI[idx][0]
pred = image2mask_network_trainerNBRFI.test_model(image=image,plot_pred=True)

- Only BBRFI

In [14]:
from pulsar_simulation.generate_data_pipeline import generate_example_payloads_for_training

#: Run this node only if training and test data needs to be generated
if  need_data_generation:
    generate_example_payloads_for_training(tag='train_only_BBRFI_v0_',
                                        num_payloads=1000,
                                        plot_a_example=False,
                                        param_folder='./syn_data/runtime/',
                                        payload_folder='./syn_data/payloads/',
                                        num_cpus=10, #: choose based on the number of nodes/cores in your system,
                                        prob_bbrfi=1,
                                        prob_nbrfi=0,                                       
                                        antenna_sensitivity=100
                                        )

    generate_example_payloads_for_training(tag='test_only_BBRFI_v0_',
                                        num_payloads=1000,
                                        plot_a_example=False,
                                        param_folder='./syn_data/runtime/',
                                        payload_folder='./syn_data/payloads/',
                                        num_cpus=10, #: choose based on the number of nodes/cores in your system,
                                        prob_bbrfi=1,
                                        prob_nbrfi=0,                                       
                                        antenna_sensitivity=100
                                        )

- Set up the datasets for training

In [ ]:
image_tag='train_only_BBRFI_v0_*_payload_detected.json' #: '*' in the name is the index place holder of a image in the image set
image_directory='./syn_data/payloads/'

mask_tag = 'train_only_BBRFI_v0_*_payload_flux.json'
mask_directory='./syn_data/payloads/'

image_mask_train_datasetBBRFI = ImageToMaskDataset(
                        image_tag = image_tag,
                        mask_tag= mask_tag,
                        image_directory = image_directory,
                        mask_directory = mask_directory,
                        image_engine=image_preprocessing_engine,
                        mask_engine=mask_preprocessing_engine
                        )

image_tag='test_only_BBRFI_v0_*_payload_detected.json' #: '*' in the name is the index place holder of a image in the image set
image_directory='./syn_data/payloads/'

mask_tag = 'test_only_BBRFI_v0_*_payload_flux.json'
mask_directory='./syn_data/payloads/'

image_mask_test_datasetBBRFI = ImageToMaskDataset(
                        image_tag = image_tag,
                        mask_tag= mask_tag,
                        image_directory = image_directory,
                        mask_directory = mask_directory,
                        image_engine=image_preprocessing_engine,
                        mask_engine=mask_preprocessing_engine
                        )
#: Plot an example
idx = 9
image_mask_train_datasetBBRFI.plot(index=idx)
image_mask_test_datasetBBRFI.plot(index=idx)

- instantiate the trainer

In [16]:
from src.pulsar_analysis.neural_network_models import  WeightedBCELoss
store_trained_model_image2mask_atBBRFI = './syn_data/model/trained_UNet_test_only_BBRFI_v0.pt'
image2mask_network_trainerBBRFI = TrainImageToMaskNetworkModel(
                                model=UNet(),
                                num_epochs=10,
                                store_trained_model_at=store_trained_model_image2mask_atBBRFI,
                                loss_criterion = WeightedBCELoss(pos_weight=3,neg_weight=1)                                
                                )

- train

In [17]:
if need_training:image2mask_network_trainerBBRFI(image_mask_pairset=image_mask_train_datasetBBRFI)

- Test

In [ ]:
idx = 0
image = image_mask_test_datasetNBRFI[idx][0]
pred = image2mask_network_trainerNBRFI.test_model(image=image,plot_pred=True)

- Define pipelines

In [19]:
ppl2pulsar = PipelineImageToCCtoLabels(image_to_mask_network=UNet(),
                                trained_image_to_mask_network_path=store_trained_model_image2mask_atPulsar,
                                min_cc_size_threshold=5
                                )

ppl2NBRFI = PipelineImageToCCtoLabels(image_to_mask_network=UNet(),
                                trained_image_to_mask_network_path=store_trained_model_image2mask_atNBRFI,
                                min_cc_size_threshold=5
                                )

ppl2BBRFI = PipelineImageToCCtoLabels(image_to_mask_network=UNet(),
                                trained_image_to_mask_network_path=store_trained_model_image2mask_atBBRFI,
                                min_cc_size_threshold=5
                                )



- generate mixed data for testing

In [20]:
from pulsar_simulation.generate_data_pipeline import generate_example_payloads_for_training
if  need_data_generation:
    generate_example_payloads_for_training(tag='test_mixed_v0_',
                                        num_payloads=100,
                                        plot_a_example=False,
                                        param_folder='./syn_data/runtime/',
                                        payload_folder='./syn_data/payloads/',
                                        num_cpus=10, #: choose based on the number of nodes/cores in your system,
                                        prob_bbrfi=0.5,
                                        prob_nbrfi=0.5
                                        )

- make test datasets

In [21]:
from src import ImageReader,ImageDataSet,LabelDataSet, LabelReader
image_tag='test_mixed_v0_*_payload_detected.json' #: '*' in the name is the index place holder of a image in the image set
image_directory='./syn_data/payloads/'

mask_tag = 'test_mixed_v0_*_payload_flux.json'
mask_directory='./syn_data/payloads/'

im_set = ImageDataSet(image_tag=image_tag,image_directory=image_directory,image_reader_engine=ImageReader(do_average=True))
m_set = ImageDataSet(image_tag=mask_tag,image_directory=mask_directory,image_reader_engine=ImageReader(do_average=True,do_binarize=True))
label_set = LabelDataSet(image_tag=image_tag,image_directory=image_directory,label_reader_engine=LabelReader())

- Mixing

In [ ]:
tn = Tuner(sample_of_objects=[ppl2pulsar,ppl2NBRFI,ppl2BBRFI],reset_components=False)

In [ ]:
ppl2mixed = tn()
ppl2mixed.display_results_in_batch(image_data_set=im_set,mask_data_set=m_set,label_data_set=label_set,randomize=False,ids_toshow=[9,7])